In [101]:
# one needs to import those packages which are needed; best to be done at the beginning of the program.
import networkx as nx
import networkx.algorithms.community as nx_comm
import numpy as np
import pandas as pd
import scipy as sp

# some basic settings for plotting figures
import matplotlib.pyplot as plt
%matplotlib inline 
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 32}

plt.rc('font', **font)
import community as community_louvain

In [2]:
G0 = nx.read_weighted_edgelist("4932.protein.links.v11.5.txt",comments="#",nodetype=str)

In [3]:
threshold_score = 400
for edge in G0.edges: 
    weight = list(G0.get_edge_data(edge[0],edge[1]).values())
    if(weight[0] <= threshold_score):
        G0.remove_edge(edge[0],edge[1])

In [4]:
#number of nodes of network
print('number of nodes of G0:',G0.number_of_nodes())

#number of edges of network
print('number of edges of G0:',G0.number_of_edges())

number of nodes of G0: 6394
number of edges of G0: 282074


In [5]:
print('Is the full G0 connected?',nx.connected.is_connected(G0))
print('How many connected subgraphs are there?',nx.connected.number_connected_components(G0))

Is the full G0 connected? False
How many connected subgraphs are there? 282


In [6]:
#get the largest component
largest_cc = max(nx.connected_components(G0),key=len)
G = G0.subgraph(largest_cc)
print('Type',type(largest_cc))
print('number of nodes of largest connected subgraph of G:',G.number_of_nodes())
print('number of edges of largest connected subgraph of G0:',G.number_of_edges())

Type <class 'set'>
number of nodes of largest connected subgraph of G: 6113
number of edges of largest connected subgraph of G0: 282074


In [11]:
ess=pd.read_csv("essential_pro.csv",header=None)

In [18]:
ess_pro=pd.Series.to_list(ess[1])

In [19]:
for i in range(len(ess_pro)):
    ess_pro[i]='4932.'+ess_pro[i]

In [21]:
#ess_pro

In [22]:
G0.remove_nodes_from(ess_pro)

In [23]:
#number of nodes of network
print('number of nodes of G0:',G0.number_of_nodes())

#number of edges of network
print('number of edges of G0:',G0.number_of_edges())

number of nodes of G0: 5098
number of edges of G0: 137012


In [25]:
nodes = nx.shortest_path(G0,'4932.YKL126W').keys()

In [100]:
G=G0.subgraph(nodes)

In [27]:
#number of nodes of network
print('number of nodes of G:',G.number_of_nodes())

#number of edges of network
print('number of edges of G:',G.number_of_edges())

number of nodes of G: 4827
number of edges of G: 137012


In [54]:
partLouvain = community_louvain.best_partition(G)
number_of_communities = max(partLouvain.values())+1 #We add one because the indexing starts at 0.
print('# of partitions for Louvain modularity =',number_of_communities)

# of partitions for Louvain modularity = 10


In [55]:
community_louvain.modularity(partLouvain, G)

0.5467413903190361

In [85]:
def community_collector(comm):
    number_of_communities = max(comm.values())+1
    communities = {} #empty dictionary
    for i in range(number_of_communities):
        communities[i] = [] #create an empty list for each community

    for name, community in comm.items():
        communities[community].append(name) 
    return communities

In [89]:
# Let's construct a dictionary object called 'communities'. The keys will be the community labels and the values 
# will be a list of nodes in that community. The more experienced python users among you will probably see an 
# easier/faster way to do this.

communities = community_collector(partLouvain)
    

# The dictionary we have constructed is similar to what the output of the Louvain algorithm in NetworkX would be. 
# In your own investigations you can decide what is more useful.

#Now let's find out how big each community is. You could accomplish this in the following way:
for k in communities:
    print('The size of community #', list(communities.keys())[k], 'is ',len(communities[k]))

The size of community # 0 is  643
The size of community # 1 is  171
The size of community # 2 is  576
The size of community # 3 is  325
The size of community # 4 is  411
The size of community # 5 is  664
The size of community # 6 is  844
The size of community # 7 is  658
The size of community # 8 is  377
The size of community # 9 is  158


In [57]:
index_1=partLouvain['4932.YKL126W']

In [58]:
sub_1=G.subgraph(communities[index_1])

In [59]:
len(sub_1.nodes)

664

In [38]:
def max_keys(dict):
    max_value=max(dict.values())
    return [k for k,v in dict.items() if v == max_value]

In [39]:
def max_key_value(dict):
   key=max_keys(dict)[0]
   return dict[key]

In [60]:
deg_cen=nx.degree_centrality(sub_1)
bet_cen = nx.betweenness_centrality(sub_1)
eig_cen = nx.eigenvector_centrality(sub_1)

In [61]:
central_dict={"Degree cen":(max_keys(deg_cen),max_key_value(deg_cen)),"Betweeness cen":(max_keys(bet_cen),max_key_value(bet_cen)),"Eigenvector cen":(max_keys(eig_cen),max_key_value(eig_cen))}

In [62]:
central_dict

{'Degree cen': (['4932.YLR113W'], 0.23529411764705882),
 'Betweeness cen': (['4932.YPL031C'], 0.039171034057359846),
 'Eigenvector cen': (['4932.YLR113W'], 0.16421691205196265)}

In [64]:
len(dendrogram)

3

In [69]:
dendrogram

[{'4932.Q0010': 2,
  '4932.YPL135W': 98,
  '4932.Q0017': 2,
  '4932.YOL103W': 1,
  '4932.YJR122W': 98,
  '4932.YGL043W': 53,
  '4932.YDR020C': 53,
  '4932.Q0143': 2,
  '4932.YBR031W': 13,
  '4932.Q0045': 73,
  '4932.Q0105': 73,
  '4932.Q0092': 2,
  '4932.Q0160': 81,
  '4932.Q0182': 2,
  '4932.Q0297': 2,
  '4932.YDR012W': 13,
  '4932.YDL020C': 89,
  '4932.YDL114W': 98,
  '4932.YLL027W': 98,
  '4932.Q0142': 2,
  '4932.YBL091C': 13,
  '4932.Q0032': 2,
  '4932.YPR067W': 98,
  '4932.Q0085': 73,
  '4932.YPL069C': 18,
  '4932.YML063W': 13,
  '4932.YLR089C': 46,
  '4932.YKL211C': 57,
  '4932.YLR441C': 13,
  '4932.YKL001C': 57,
  '4932.YKL149C': 63,
  '4932.YIL099W': 46,
  '4932.YAL058W': 18,
  '4932.YNR064C': 52,
  '4932.YLL052C': 45,
  '4932.YML120C': 73,
  '4932.YOR187W': 13,
  '4932.YDL202W': 13,
  '4932.YMR287C': 73,
  '4932.YPR020W': 73,
  '4932.YPL172C': 73,
  '4932.YJL116C': 73,
  '4932.YGR033C': 73,
  '4932.YKL155C': 13,
  '4932.Q0055': 73,
  '4932.YGR021W': 73,
  '4932.YGL226W': 73,
 

In [93]:
dendrogram = community_louvain.generate_dendrogram(G)
# for level in range(1,len(dendrogram) - 1) :
#      print("partition at level", level, "is", community_louvain.partition_at_level(dendrogram, level))  # NOQA

In [95]:

# Let's construct a dictionary object called 'communities'. The keys will be the community labels and the values 
# will be a list of nodes in that community. The more experienced python users among you will probably see an 
# easier/faster way to do this.

communities = community_collector(community_louvain.partition_at_level(dendrogram, 0))
    

# The dictionary we have constructed is similar to what the output of the Louvain algorithm in NetworkX would be. 
# In your own investigations you can decide what is more useful.

#Now let's find out how big each community is. You could accomplish this in the following way:
for k in communities:
    print('The size of community #', list(communities.keys())[k], 'is ',len(communities[k]))

The size of community # 0 is  274
The size of community # 1 is  12
The size of community # 2 is  3
The size of community # 3 is  6
The size of community # 4 is  33
The size of community # 5 is  19
The size of community # 6 is  99
The size of community # 7 is  434
The size of community # 8 is  30
The size of community # 9 is  9
The size of community # 10 is  4
The size of community # 11 is  22
The size of community # 12 is  18
The size of community # 13 is  8
The size of community # 14 is  20
The size of community # 15 is  5
The size of community # 16 is  13
The size of community # 17 is  4
The size of community # 18 is  59
The size of community # 19 is  2
The size of community # 20 is  2
The size of community # 21 is  8
The size of community # 22 is  11
The size of community # 23 is  9
The size of community # 24 is  2
The size of community # 25 is  5
The size of community # 26 is  6
The size of community # 27 is  26
The size of community # 28 is  4
The size of community # 29 is  33
The

In [96]:
index_1=community_louvain.partition_at_level(dendrogram, 0)['4932.YKL126W']

In [98]:
len(communities[index_1])

434

In [102]:
aa=nx_comm.louvain_partitions(G, weight='weight', resolution=2)

In [103]:
aa

<generator object louvain_partitions at 0x0000022101D50430>

In [125]:
nxLouvain=nx_comm.louvain_communities(G, seed=123, resolution=5)
len(nxLouvain)

70

In [126]:
idx='x'
for i in range(len(nxLouvain)):
    #print('Community', i, 'has', len(nxLouvain[i]), 'nodes.')
    if '4932.YKL126W' in nxLouvain[i]:
        idx=i

In [127]:
idx

25

In [128]:
len(nxLouvain[idx])

34